In [1]:
%matplotlib qt
%load_ext autoreload
%autoreload 2

# Pitching Airfoil 

In [2]:
import jax.numpy as jnp
import numpy as np
import matplotlib.pyplot as plt
from functools import partial
from jax import jit
import jax

In [3]:
from ICARUS.Systems.second_order_system import SecondOrderSystem
from ICARUS.Systems.integrate import BackwardEulerIntegrator, ForwardEulerIntegrator, RK4Integrator, RK45Integrator, CrankNicolsonIntegrator, GaussLegendreIntegrator, NewmarkIntegrator

# Node

In [5]:
from enum import Enum

class ValueType(Enum):
    VALUE = 0
    DERIVATIVE = 1
    ACCELERATION = 2

    def __str__(self):
        return self.name
    
class GlobalCoordinateSystem(Enum):
    X = 0       # Position of the node in the x direction
    Y = 1       # Position of the node in the y direction
    Z = 2       # Position of the node in the z direction
    XZ = 3      # Angle of the node in the xz plane
    YZ = 4      # Angle of the node in the yz plane
    XY = 5      # Angle of the node in the xy plane
    OTHER = 6   # Other

    def __str__(self):
        return self.name

class BoundaryCondition(Enum):
    FIXED = 0
    FREE = 1

    def __str__(self):
        return self.name

class DOFType:
    """
    A degree of freedom (DOF) type combines the type of variable with the type of value it represents.
    """
    __slots__ = ['value_type', 'coordinate_system']
    def __init__(self, value_type: ValueType, coordinate_system: GlobalCoordinateSystem):
        self.value_type = value_type
        self.coordinate_system = coordinate_system

    def __str__(self):
        return f"{self.value_type} {self.coordinate_system}"

class DOF:
    idx = 0
    var_global_idx = {}
    """
    A degree of freedom (DOF) represents a single variable that can be solved for in the system.
    Variables can be displacements, velocities, accelerations, etc.
    A variable has these properties:
        - A name that identifies the variable.
        - A global index that identifies the variable in the global system of equations.
        - A boundary condition that can be applied to the variable.
        - A value that represents the value of the variable.
        - A local perturbation that can be applied to the variable.
    """
    def __init__(
        self,
        name: str, 
        value: float, 
        type: DOFType, 
        boundary_condition: BoundaryCondition = BoundaryCondition.FREE
    ):
        self.name = name
        self.value = value
        self.dvalue = 0.0
        self.ddvalue = 0.0
        self.type = type
        self.boundary_condition = boundary_condition

        # Assign a global index if it is not already assigned
        if name not in DOF.var_global_idx:
            self.index = DOF.idx
            DOF.idx += 1
            DOF.var_global_idx[name] = len(DOF.var_global_idx)

    @staticmethod
    def get_global_idx(key)-> int:
        return DOF.var_global_idx[key]
    
    @staticmethod
    def get_global_dof_name(idx):
        for key, value in DOF.var_global_idx.items():
            if value == idx:
                return key
        return None
    
    @staticmethod
    def get_global_dofs():
        items = list(DOF.var_global_idx.keys())
        items.sort(key = lambda x: DOF.var_global_idx[x])
        for key in items:
            print(f"Global DOF: {key} -> {DOF.var_global_idx[key]}")


class Node:
    idx = 0
    """
    A node represents a point that has some degree of freedom (DOF) associated with it.
    Additionally it can have boundary conditions applied to it.
    """
    def __init__(self, degree_of_freedom: list[DOF],):
        self.dof = len(degree_of_freedom)
        self.variables = degree_of_freedom
        self.boundary_conditions = {}
        for var in degree_of_freedom:
            self.boundary_conditions[var.name] = var.boundary_condition


        # Node index
        self.index = Node.idx
        Node.idx += 1  

    def __str__(self):
        string =  f"Node {self.index} with {self.dof} degrees of freedom\n"
        for var in self.variables:
            string += f"\t{var.type}: {var.name} -> {var.value}, {var.dvalue}\n"
        return string

X_DISPLACEMENT = DOFType(ValueType.VALUE, GlobalCoordinateSystem.X)
Y_DISPLACEMENT = DOFType(ValueType.VALUE, GlobalCoordinateSystem.Y)
Z_DISPLACEMENT = DOFType(ValueType.VALUE, GlobalCoordinateSystem.Z)
THETA_ROTATION = DOFType(ValueType.VALUE, GlobalCoordinateSystem.XZ)
PHI_ROTATION = DOFType(ValueType.VALUE, GlobalCoordinateSystem.YZ)
PSI_ROTATION = DOFType(ValueType.VALUE, GlobalCoordinateSystem.XY)

x1 = DOF("x1", 0.0, X_DISPLACEMENT, BoundaryCondition.FIXED)
x2 = DOF("x2", 0.0, X_DISPLACEMENT, BoundaryCondition.FREE)
x3 = DOF("x3", 0.0, X_DISPLACEMENT, BoundaryCondition.FREE)

y1 = DOF("y1", 0.0, Y_DISPLACEMENT, BoundaryCondition.FIXED)
y2 = DOF("y2", 0.0, Y_DISPLACEMENT, BoundaryCondition.FREE)
y3 = DOF("y3", 0.0, Y_DISPLACEMENT, BoundaryCondition.FREE)

theta1 = DOF("theta1", 0.0, THETA_ROTATION, BoundaryCondition.FIXED)
theta2 = DOF("theta2", 0.0, THETA_ROTATION, BoundaryCondition.FREE)
theta3 = DOF("theta3", 0.0, THETA_ROTATION, BoundaryCondition.FREE)

DOF.get_global_dofs()

# Create a simple 3 node system
node1 = Node([x1, y1, theta1])
node2 = Node([x2, y2, theta2])
node3 = Node([x3, y3, theta3])

print("\nExample Node 1:")
print(node1)

Global DOF: x1 -> 0
Global DOF: x2 -> 1
Global DOF: x3 -> 2
Global DOF: y1 -> 3
Global DOF: y2 -> 4
Global DOF: y3 -> 5
Global DOF: theta1 -> 6
Global DOF: theta2 -> 7
Global DOF: theta3 -> 8

Example Node 1:
Node 0 with 3 degrees of freedom
	VALUE X: x1 -> 0.0, 0.0
	VALUE Y: y1 -> 0.0, 0.0
	VALUE XZ: theta1 -> 0.0, 0.0



# Finite Element

In [6]:
class FiniteElement:
    """
    A finite element is a local approximation of a continuous system. It uses basis functions to approximate the
    continuous system between the nodes. When the nodes move, the finite element moves with them.
    """
    def __init__(self, *args, **kwargs):
        self.variables: list[DOF] = []
        self.nodes: list[Node] = []
        self.dof: int = 0
        self.global_indices: list[int] = []
        raise NotImplementedError("FiniteElement class must be implemented")

    def M(self, *args, **kwargs):
        """
        Represents the mass matrix of the finite element
        """
        raise NotImplementedError("M method must be implemented")

    def C(self, *args, **kwargs):
        """
        Represents the damping matrix of the finite element
        """
        raise NotImplementedError("C method must be implemented")

    def f_int(self, *args, **kwargs):
        """
        Represents the internal force of the finite element
        """
        raise NotImplementedError("f_int method must be implemented")
    
    def f_ext(self, *args, **kwargs):
        """
        Represents the external force of the finite element
        """
        raise NotImplementedError("f_ext method must be implemented")

    @property
    def R(self):
        """
        Represents the transformation matrix of the finite element to the global coordinate system
        """
        raise NotImplementedError("R property must be implemented")
    
    @property
    def N(self):
        """
        Represents the shape (basis) functions of the finite element.
        """
        raise NotImplementedError("N method must be implemented")

# Beam Element

In [61]:
class Beam:
    __slots__ = ["E", "I", "A", "L", "rho"]
    def __init__(self, E,I, A, L, rho):
        self.E = E
        self.A = A
        self.L = L
        self.rho = rho
        self.I = I

class Beam2DFiniteElement(FiniteElement):
    """
    A 2D beam finite element.
    It models the behaviour of a 2D beam in a plane by 
    A 2D beam can be used to model a 2D beam in a plane. It has 4 degrees of freedom per node:
        - 2 displacements (x, y)
        - 1 rotations (θ)

    In total, a 2D beam has 6 degrees of freedom:
        - 2 displacements (x, y) for node 1
        - 1 rotations (θ) for node 1
        - 2 displacements (x, y) for node 2
        - 1 rotations (θ) for node 2
    """
    def __init__(self, beam: Beam, node_1: Node, node_2: Node):
        self.E = beam.E
        self.A = beam.A
        self.L = beam.L
        self.I = beam.I
        self.rho = rho
        
        # Check that the nodes are valid
        if not isinstance(node_1, Node) or not isinstance(node_2, Node):
            raise ValueError("node_1 and node_2 must be instances of the Node class")
        
        self.node_1 = node_1
        self.node_2 = node_2
        self.nodes = [node_1, node_2]
        
        # Register the finite element degrees of freedom
        dof = {}
        self.variables = []
        i = 0
        for degree in self.node_1.variables:
            # Add the degree to the dict if not already present
            dof[degree.name] = i
            self.variables.append(degree)
            i+=1

        for degree in self.node_2.variables:
            if degree not in dof.keys():
                dof[degree.name] = i
                self.variables.append(degree)
                i+=1

        self.dof = len(dof)

        # Note the global indices of the degrees of freedom
        self.global_indices = [DOF.var_global_idx[d] for d in dof]

    def calculate_basis_functions(self, y):
        return None
       

    
    def N(self, y):
        """
        Shape functions
        They are calculated in the local coordinate system by the virtual work principle. Given the boundary conditions.
        For a beam element the boundary conditions are:
            - At the start of the beam (x = 0), the displacements and rotations are 0.
            - At the end of the beam (x = L), the displacements and rotations are 0. 
        This means that we have 6 basis functions for the beam element.

        We want to solve the system N(0) * u = 0 and N(L) * u = 0 where u is the vector of displacements and rotations.
        The general form of the shape functions is:
        N1 = a + by + cy^2 + dy^3 ... of degree node.dof
        """
        L = self.L
        var = y/L
        raise NotImplementedError("N method must be implemented")

    def local_M(self,t,x):
        """
        Inertia matrix of the finite element in local coordinates.
        It is calculated by integrating the mass density over the element using the basis functions N.
        M = ∫(N^T * ρA * N) dy
        """
        integral = lambda y: self.N(y).T @ self.rho * self.A * self.N(y)
        # Integrate the inertia matrix over the length of the beam
        return jax.scipy.integrate.trapezoid(integral, 0, self.L)
        
    
    def local_K(self,t,x):
        """
        Stiffness matrix of the finite element in local coordinates.
        It is calculated by integrating the stiffness over the element using the basis functions N.
        K = ∫(N^T'' * EIxx * N'') dy
        """
        integral = lambda y: self.N(y).T @ self.E * self.I * self.N(y)
        raise jax.scipy.integrate.trapezoid(integral, 0, self.L)
        
    
    def local_C(self,t,x):
        """
        Damping matrix of the finite element in local coordinates.
        It is calculated by integrating the damping over the element using the basis functions N.
        C = ∫(N^T * η * N') dy
        """
        return jnp.zeros((self.dof, self.dof))
    
    @property
    def R(self):
        """
        For a beam element, the transformation matrix is the rotation matrix of the element.
        """
        node1 = self.node_1
        node2 = self.node_2

        y2 = node2.variables["y2"].value
        y1 = node1.variables["y1"].value
        x2 = node2.variables["x2"].value
        x1 = node1.variables["x1"].value

        phi = np.arctan2(y2 - y1, x2 - x1)
        return jnp.array([
            [jnp.cos(phi), jnp.sin(phi), 0, 0, 0, 0],
            [-jnp.sin(phi), jnp.cos(phi), 0, 0, 0, 0],
            [0, 0, 1, 0, 0, 0],
            [0, 0, 0, jnp.cos(phi), jnp.sin(phi), 0],
            [0, 0, 0, -jnp.sin(phi), jnp.cos(phi), 0],
            [0, 0, 0, 0, 0, 1]
        ])
    
    @partial(jit, static_argnums=(0))
    def M(self,t,x):
        return self.R.T @ self.local_M(t,x) @ self.R
    
    @partial(jit, static_argnums=(0))
    def K(self,t,x):
        return self.R.T @ self.local_K(t,x) @ self.R

    @partial(jit, static_argnums=(0)) 
    def C(self,t,x):
        return self.R.T @ self.local_C(t,x) @ self.R
    
    def __str__(self):
        return f"Finite Element with {self.dof} degrees of freedom.\n\tNode 1: {self.node_1}\n\tNode 2: {self.node_2}"

# # Connect node 1 to node 2
# # Compute the angle of the connection
# phi = np.arctan2(node2.variables["y2"] - node1.variables["y1"], node2.variables["x2"] - node1.variables["x1"])
# conn1 = NodeConnection(node1, node2, phi)

# # Connect node 2 to node 3
# # Compute the angle of the connection
# phi = np.arctan2(node3.variables["y3"] - node2.variables["y2"], node3.variables["x3"] - node2.variables["x2"])
# conn2 = NodeConnection(node2, node3, phi)

# print(conn2.variables)
# print(conn2.global_indices)
# print(conn1.variables)
# print(conn1.global_indices)

# Create two finite elements
E = 235e9
A = np.pi * (0.3**2) / 4
L = 2.0
I = 30e-2
rho = 5490

beam1 = Beam(E, A, L, I, rho)
beam2 = Beam(E, A, L, I, rho)
element1 = Beam2DFiniteElement(beam1, node1, node2)
element2 = Beam2DFiniteElement(beam2, node2, node3)

In [13]:
class NodalSystem:
    """
    An nodal system represents a collection of finite elements that are connected to each other. 
    """
    def __init__(self, elements: list[FiniteElement]):
        self.elements: list[FiniteElement] = elements
        
        # Register the degrees of freedom using a set
        dof = {}
        for element in elements:
            for variable in element.variables:
                dof[variable] = DOF.get_global_idx(variable)
        self.variables = dof
        self.dof = len(dof)

    @partial(jit, static_argnums=(0))
    def M(self,t, x):
        # Construct the M matrix by adding the local M matrices. 
        # The global M matrix is a block matrix of the local M matrices
        # The matrix should be sparse
        M = jnp.zeros((self.dof, self.dof))

        # To construct the matrix loop over all the elements. 
        for element in self.elements:
            # Get each elements K matrix expressed in the global coordinate system
            sub_M =  element.M(t,x)
            for local_row, dof in enumerate(element.variables):
                # Set the mask true for each idx in the Nodes global indices
                global_row = DOF.get_global_idx(dof)
                columns = jnp.array([ i  for i in element.global_indices])          
                M = M.at[global_row, columns].add(sub_M[local_row, :])
        # print(f"M: ")
        # print(M)
        return M
    
    @partial(jit, static_argnums=(0))
    def K(self, t, x):
        # Construct the K matrix by adding the local K matrices. 
        # The global K matrix is a block matrix of the local K matrices
        # The matrix should be sparse
        K = jnp.zeros((self.dof, self.dof))

        # To construct the matrix loop over all the elements. 
        for element in self.elements:
            # Get each elements K matrix expressed in the global coordinate system
            sub_K =  element.f_int(t,x)
            for local_row, dof in enumerate(element.variables):
                # Set the mask true for each idx in the Nodes global indices
                global_row = DOF.get_global_idx(dof)
                columns = jnp.array([ i  for i in element.global_indices])
                K = K.at[global_row, columns].add(sub_K[local_row,:])
        # print(f"K: ")
        # print(K)
        return K
    
    @partial(jit, static_argnums=(0))
    def C(self,t,x):
        # Construct the C matrix by adding the local C matrices. 
        # The global C matrix is a block matrix of the local C matrices
        # The matrix should be sparse
        C = jnp.zeros((self.dof, self.dof))

        # To construct the matrix loop over all the elements. 
        for element in self.elements:
            # Get each elements C matrix expressed in the global coordinate system
            sub_C =  element.C(t,x)
            for node in element.nodes:
                for local_row, dof in enumerate(node.variables):
                    # Set the mask true for each idx in the Nodes global indices
                    global_row = DOF.get_global_idx(dof)                    
                    columns = jnp.array([ i  for i in element.global_indices])               
                    C = C.at[global_row, columns].add(sub_C[local_row, :])
        # print(f"C:")
        # print(C)
        return C        
    
# Create the system
sys = NodalSystem([element1, element2])

def f_ext(t: float, x: jnp.ndarray) -> jnp.ndarray:
    return jnp.array([
        jnp.cos(5*t),                       # x1
        0.0,                                # y1
        0.0,                                # theta1
        -jnp.cos(5*t)/2,                    # x2
        0.0,                                # y2
        0.0,                                # theta2
        -jnp.cos(5*t)/2,                    # x3    
        0.0,                                # y3
        0.0                                 # theta3,
    ])

# Model

In [14]:
system = SecondOrderSystem(sys.M, sys.C, sys.K, f_ext)
x0 = jnp.zeros(sys.dof*2)
# x0 = x0.at[0].set(0.1)

A = system.linearize(0.0, x0)[0]
M = sys.M(0.0, x0)
C = sys.C(0.0, x0)
K = sys.K(0.0, x0)

# # Plot the matrices
# ax = plt.matshow(A)
# plt.title("A matrix")
# # Add text to each cell
# for i in range(sys.dof*2):
#     for j in range(sys.dof*2):
#         num = A[i,j]
#         if num == 0:
#             continue
#         # Convert num to scientific notation with 3 decimal points
#         num = f"{num:.1e}"
#         plt.text(j, i, num, ha='center', va='center', color='white', fontsize=4)

# plt.show()

# ax = plt.matshow(M)
# plt.title("M matrix")
# # Add text to each cell
# for i in range(sys.dof*2):
#     for j in range(sys.dof*2):
#         num = M[i,j]
#         if num == 0:
#             continue
#         # Convert num to scientific notation with 3 decimal points
#         num = f"{num:.1e}"
#         plt.text(j, i, num, ha='center', va='center', color='white', fontsize=4)

# plt.show()

# ax = plt.matshow(C)
# plt.title("C matrix")
# # Add text to each cell
# for i in range(sys.dof*2):
#     for j in range(sys.dof*2):
#         num = C[i,j]
#         if num == 0:
#             continue
#         # Convert num to scientific notation with 3 decimal points
#         num = f"{num:.1e}"
#         plt.text(j, i, num, ha='center', va='center', color='white', fontsize=4)

# plt.show()

# ax = plt.matshow(K)
# plt.title("K matrix")
# # Add text to each cell
# for i in range(sys.dof*2):
#     for j in range(sys.dof*2):
#         num = K[i,j]
#         if num == 0:
#             continue
#         # Convert num to scientific notation with 3 decimal points
#         num = f"{num:.1e}"
#         plt.text(j, i, num, ha='center', va='center', color='white', fontsize=4)

# plt.show()

system.eigenvalues(0.0, x0)

Array([-7.5275992e+04+0.0000000e+00j,  8.0566406e-03+8.2213742e+04j,
        8.0566406e-03-8.2213742e+04j, -5.9605230e+04+0.0000000e+00j,
       -3.5400391e-03+6.7198289e+04j, -3.5400391e-03-6.7198289e+04j,
        7.5275930e+04+0.0000000e+00j,  5.9605277e+04+0.0000000e+00j,
       -1.3427734e-03+1.1332051e+04j, -1.3427734e-03-1.1332051e+04j,
        2.6321411e-04+5.6660239e+03j,  2.6321411e-04-5.6660239e+03j,
        5.7983398e-03+1.9356770e+04j,  5.7983398e-03-1.9356770e+04j,
       -7.0289154e+00+5.3583355e+00j, -7.0289154e+00-5.3583355e+00j,
        7.0313339e+00+5.3407125e+00j,  7.0313339e+00-5.3407125e+00j],      dtype=complex64)

In [15]:
jnp.linalg.inv(sys.elements[0].M(0.0, x0))

Array([[ 0.00515377,  0.        ,  0.        , -0.00257689,  0.        ,
         0.        ],
       [ 0.        ,  0.02061507, -0.07730651,  0.        , -0.00515375,
        -0.03865322],
       [ 0.        , -0.07730649,  0.3865325 ,  0.        ,  0.03865318,
         0.2705726 ],
       [-0.00257689,  0.        ,  0.        ,  0.00515377,  0.        ,
         0.        ],
       [-0.        , -0.00515375,  0.03865319, -0.        ,  0.02061507,
         0.07730648],
       [ 0.        , -0.03865319,  0.2705725 ,  0.        ,  0.07730648,
         0.3865324 ]], dtype=float32)

In [16]:
dt0 = 1e-5
t0 = 0.0
t_end = 60

# test_all_integrators(system, x0, t0, t_end, dt0, compare_with_scipy = False)

# # Newmark integrator
newmark = NewmarkIntegrator(dt0, system, gamma=0.5, beta=0.25)

t_data, x_data = newmark.simulate(x0, t0, t_end)
# t_data, x_data = gauss_legendre.simulate(x0, t0, t_end)

In [17]:
import matplotlib.pyplot as plt

clip = 1000


fig, ax = plt.subplots(3, 1, figsize=(10, 10))

# Clip the data to only sample 1000 points
clip = jnp.maximum(1, int(len(t_data) / 1000))

ax[0].set_title("Displacement of node 1")
ax[0].plot(t_data[::clip], x_data[:,0][::clip], label="x1")
ax[0].plot(t_data[::clip], x_data[:,1][::clip], label="y1")
ax[0].plot(t_data[::clip], x_data[:,2][::clip], label="theta1")
ax[0].legend()

ax[1].set_title("Displacement of node 2")
ax[1].plot(t_data[::clip], x_data[:,3][::clip], label="x2")
ax[1].plot(t_data[::clip], x_data[:,4][::clip], label="y2")
ax[1].plot(t_data[::clip], x_data[:,5][::clip], label="theta2")
ax[1].legend()

ax[2].set_title("Displacement of node 3")
ax[2].plot(t_data[::clip], x_data[:,6][::clip], label="x3")
ax[2].plot(t_data[::clip], x_data[:,7][::clip], label="y3")
ax[2].plot(t_data[::clip], x_data[:,8][::clip], label="theta3")
ax[2].legend()